# 反向传播的实现

- 根据链式法则求所有变量的偏导（一元内相乘，多元间相加）
  $$
    \frac{\partial L}{\partial x} = \sum_i \frac{\partial L}{\partial y_i}\frac{\partial y_i}{\partial x}
  $$
- 局部梯度的计算需要知道区分运算，还需要知道运算数
- 必须自己定义一个类用运算符重载和属性实现这些功能（Value）

## 反向传播类

In [1]:
from backward import Value

## 计算图绘制函数

In [2]:
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._child:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='pdf', rankdir='LR', filename='computation_graph'):
    """
    输出计算图为 PDF 文件
    :param root: 计算图根节点（如 loss Value 对象）
    :param format: 输出格式，设为 'pdf' 即可
    :param rankdir: 布局方向（LR=左右，TB=上下）
    :param filename: 输出文件名（无需加 .pdf 后缀）
    :return: Digraph 对象，调用 render() 生成文件
    """
    print(f"Drawing graph in format: {format}, rankdir: {rankdir}")
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    print(f"Nodes: {len(nodes)}, Edges: {len(edges)}")
    
    # 关键修改：format 设为 'pdf'
    dot = Digraph(
        format=format,
        graph_attr={'rankdir': rankdir},
        node_attr={'shape': 'record'},  # 固定节点形状为 record
        filename=filename  # 指定输出文件名（后续无需重复写）
    )
    
    for n in nodes:
        # 节点标签：显示 data 和 grad（保留4位小数）
        dot.node(
            name=str(id(n)),
            label=f"{{ data: {n.data:.4f} | grad: {n.grad:.4f} }}"
        )
    
    for n1, n2 in edges:
        # 绘制节点间的边（子节点 → 父节点，符合计算图方向）
        dot.edge(str(id(n1)), str(id(n2)))
    
    # 生成 PDF 文件（默认保存在当前目录）
    dot.render(filename=filename, view=False)  # view=True 会自动打开 PDF
    print(f"PDF 文件已保存为：{filename}.pdf")
    return dot

In [3]:
from backward import ActivationFunction, LossFunction, Optimizer, to_one_hot, Layer, SGDMomentum, Adam
from data_loader import load_mnist_dataset
import numpy as np
numInputs, numOutputs, numHiddens = 3, 5, 3 # 定义模型参数

L1 = Layer(numInputs, numHiddens)
L2 = Layer(numHiddens, numOutputs)

opti = Optimizer(L1.parameters()+L2.parameters(), 0.01)

# train_images, train_labels, test_images, test_labels = load_mnist_dataset('./data', flatten=True)
# train_images = np.array(train_images)
train_images = np.array([[0, 0, 4], [5, 0, 1], [1, 5, 0]])

train_labels = np.array([2, 0, 1])



lossL = [0] * 10
for epoch in range(1):
    print(f"Epoch: {epoch}")
    for imgIdx in range(1):

        img = train_images[imgIdx]

        label = train_labels[imgIdx]
        # 前向传播
        a1 = L1(img)
        # print(f"a1: {a1}")
        a2 = L2(ActivationFunction.RelU(a1))
        # print(f"a2: {a2}")
        yHat = ActivationFunction.softmax(a2)
        # print(f"yHat: {yHat}")
        oh = to_one_hot(label, 10)
        # print(oh)
        loss = LossFunction.categorical_cross_entropy(yHat, oh)
        diff = loss.data - lossL[label]
        lossL[label] = loss.data
        formatted_losses = [f"{loss:6.2f}" for loss in lossL]
        print(f"losses{label}:{diff}: {', '.join(formatted_losses)}")
        print(f"avarage loss: {np.mean(lossL)}")
        # print("Forward pass completed")
        opti.zero_grad()
        loss.backward()
        draw_dot(loss)
        print(loss)
        opti.step()

Epoch: 0
add 节点 2065752981824 的子节点：[2065752849824, 2065752850160]
add 节点 2065752982064 的子节点：[2065752981824, 2065752981632]
add 节点 2065752982304 的子节点：[2065752982064, 2065223308096]
add 节点 2065752983552 的子节点：[2065752982640, 2065752982976]
add 节点 2065752983792 的子节点：[2065752983552, 2065752983312]
add 节点 2065752984032 的子节点：[2065752983792, 2065752700304]
add 节点 2065752985280 的子节点：[2065752984368, 2065752984704]
add 节点 2065752985520 的子节点：[2065752985280, 2065752985040]
add 节点 2065752985760 的子节点：[2065752985520, 2065752225744]
add 节点 2065752987680 的子节点：[2065752986960, 2065752987200]
add 节点 2065752987920 的子节点：[2065752987680, 2065752987440]
add 节点 2065752988160 的子节点：[2065752987920, 2065752840560]
add 节点 2065752989120 的子节点：[2065752988400, 2065752988640]
add 节点 2065752989360 的子节点：[2065752989120, 2065752988880]
add 节点 2065752989600 的子节点：[2065752989360, 2065752848672]
add 节点 2065752990560 的子节点：[2065752989840, 2065752990080]
add 节点 2065752990800 的子节点：[2065752990560, 2065752990320]
add 节点 2065752991040 的

## 示例

In [4]:
x1 = Value(5)
x2 = Value(2)
loss = ((2 + x1)**2 + (4 - x2)**2) / 2
loss.backward()
# draw_dot(loss)

add 节点 2065752840224 的子节点：[2065750139264, 2065752844208]
add 节点 2065753019344 的子节点：[2065753019008, 2065753018960]
add 节点 2065753019920 的子节点：[2065753018624, 2065753019680]
后序遍历添加节点：2065750139264
后序遍历添加节点：2065752844208
后序遍历添加节点：2065752840224
后序遍历添加节点：2065752843152
后序遍历添加节点：2065753018624
后序遍历添加节点：2065752694976
后序遍历添加节点：2065753018864
后序遍历添加节点：2065753019008
后序遍历添加节点：2065753018960
后序遍历添加节点：2065753019344
后序遍历添加节点：2065753019488
后序遍历添加节点：2065753019680
后序遍历添加节点：2065753019920
后序遍历添加节点：2065753020064
后序遍历添加节点：2065753020256
正确拓扑顺序（子→父）： [2065750139264, 2065752844208, 2065752840224, 2065752843152, 2065753018624, 2065752694976, 2065753018864, 2065753019008, 2065753018960, 2065753019344, 2065753019488, 2065753019680, 2065753019920, 2065753020064, 2065753020256]
执行节点 2065753020256 的 _backward
执行节点 2065753019920 的 _backward
add 节点的 _backward 被调用！
执行节点 2065753019680 的 _backward
执行节点 2065753019344 的 _backward
add 节点的 _backward 被调用！
执行节点 2065753019008 的 _backward
执行节点 2065753018624 的 _backward
执行节点 20657528

## 线性层的实现

In [5]:
import numpy as np
class Neuron:
    def __init__(self, inNum: int) -> None:
        # 用NumPy数组存储权重（形状：(in_num,)），每个元素是Value类型
        self.weights = np.array([
            Value(np.random.uniform(-1, 1)) 
            for _ in range(inNum)
        ])
        # 偏置（单个Value）
        self.bias = Value(np.random.uniform(-1, 1))
    
    def __call__(self, x: np.ndarray) -> Value:
        # 计算线性部分：z = w·x + b
        weighted_sum = np.sum(self.weights * x) + self.bias
        return weighted_sum
    
    def parameters(self):
        '''
            [w1, w2, ..., b]
        '''
        return list(self.weights) + [self.bias]

class Layer:
    def __init__(self, inNum: int, outNum: int) -> None:
        '''
            inNum: 输入特征数
            outNum: 神经元个数
        '''
        self.neurons = np.array([Neuron(inNum) for _ in range(outNum)])
    
    def __call__(self, x: np.ndarray) -> np.ndarray:
        """
            前向传播：计算层的输出（所有神经元的输出组成的数组）
            out = verctorX @ Layer.W + Layer.b
        """
        # 对每个神经元调用__call__方法，得到输出列表后转为NumPy数组
        out = np.array([neuron(x) for neuron in self.neurons])
        return out
    
    def parameters(self):
        # 遍历每个神经元，收集参数并合并为一个列表
        return [p for neuron in self.neurons for p in neuron.parameters()]
    
    def zeroGrad(self):
        for n in self.parameters():
            n.data = 0

## 多层感知机